**[Deep Learning Home Page](https://www.kaggle.com/learn/deep-learning)**

---


Para poder entrenar el modelo con la imagen original y con la imagen especular lo hacemos con el argumento de volteo horizontal al generador de datos de imagen si establecemos un volteo horizontal a verdadero el generdor de datos de imagen decidirá aleatoriamente si voltea o no una imagen cada vez que esta a punto de enviar la imagen al modelo para capacitación: 

**data_generator_with_aug=ImagenDataGenerator(preprocessing_fuction=preprocess_input, horizontal_flip=True)**

Hay otros datos similares tecnicas de aumento enumeradas en la documentación del generador de datos de imagen que puede ver en los nucleos escribiendo generador de datos de imagen y un signo de interrogación, por ejemplo, podramos recortar ligeramente la foto desplazandola horizontal o verticalmente y agregar comendos para cambiar aleatoriamente parte de la imagen que entrenamos en un 20% mas a cada lado y un 20% mas arriba y abajo, estableciendo con el rango de cambio y el rango de cambio de altura a 0.2

**data_generator_with_aug=ImagenDataGenerator(preprocessing_fuction=preprocess_input, horizontal_flip=true, width_shift_range=0.2, height_shift_range = 0.2)**

**train_generator = data_gen_with_aug.flow_from_directory('directorio', target_size=(image_size, image_size),batch_size=6, class_mode='categorical')**

Generador de nuestros datos de validación que no hace este tipo de aumentos o manupulación de un modelo de medición de calidad, utilizará las mismas imagenes en bruto que usamos antes.

**data_gen_no_aug=ImagenDataGenerator(preprocessing_fuction=preprocess_input)**

**valid_generator=data_gen_no_aug.flow_from_directory('directorio', target_size=(image_size, image_size), class_mode='categorical', batch_size=20)**

epochs=2 significa que revisa cada archivo de imagen sin procesar dos veces con las que puede experimentar, el aumento de datos permite usar mas valor en epochs antes de comenzar a sobreajustar y ver puntiajes de validación, empeoraran en este caso particular, nuestros datos de vcalidación si son pequeños. Es mas fiable un conjunto de datos de validación mas grandes y eso es algo que vamos a ver.

**my_new_model.fit_generator(train_generator, steps_per_epoch=12, epochs=2, validacion_data=valid_generator, validation_steps=1)**


Sobreajuste: evita el ajuste de la red neuronal a una imagen particular o a un dato en particular, cuando realiza una recta de regresion esta es como la media entre los datos que tiene, sin embargo un sobreajuste seria una linea discontinua dando como resultado el ajuste a ese ejemplo en concreto. Cuando realizamos aumento de datos disminuimos el sobreajuste.



# Exercise Introduction
We will return to the automatic rotation problem you worked on in the previous exercise. But we'll add data augmentation to improve your model.
The model specification and compilation steps don't change when you start using data augmentation. The code you've already worked with for specifying and compiling a model is in the cell below.  Run it so you'll be ready to work on data augmentation.

**Introducción al ejercicio**
Volveremos al problema de rotación automática en el que trabajó en el ejercicio anterior. Pero agregaremos el aumento de datos para mejorar su modelo.

La especificación del modelo y los pasos de compilación no cambian cuando comienza a utilizar el aumento de datos. El código con el que ya trabajó para especificar y compilar un modelo se encuentra en la celda a continuación. Ejecútelo para que esté listo para trabajar en el aumento de datos.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_5 import *
print("Setup Complete")

# 1) Fit the Model Using Data Augmentation

Here is some code to set up some ImageDataGenerators. Run it, and then answer the questions below about it.

**Ajustar el modelo utilizando el aumento de datos**
Aquí hay un código para configurar algunos ImageDataGenerators. Ejecútelo y luego responda las siguientes preguntas al respecto.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

image_size = 224

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True,
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)

            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_1.solution()

¿Por qué necesitamos tanto un generador con aumento como un generador sin aumento? Después de pensarlo, mira la solución de abajo.

Queremos hacer un aumento de datos al ajustar el modelo por las razones mencionadas en el video (incluyendo una reducción en el ajuste, al darnos más datos con los que trabajar).

Pero no queremos cambiar la forma en que probamos el modelo. Así que el generador de validación usará un ImageDataGenerator sin aumento. Esto permite una comparación directa entre los diferentes procedimientos de entrenamiento (por ejemplo, entrenamiento con aumento y sin él).

Si el aumento hiciera más difícil predecir la etiqueta asociada a una imagen (por ejemplo, debido a la forma en que la imagen fue recortada en el aumento) eso haría que fuera engañoso comparar las puntuaciones con otro procedimiento en el que los datos de validación fueran sólo imágenes originales.

# 2) Choosing Augmentation Types
ImageDataGenerator offers many types of data augmentation. For example, one argument is `rotation_range`. This rotates each image by a random amount that can be up to whatever value you specify.

Would it be sensible to use automatic rotation for this problem?  Why or why not?

**Elección de tipos de aumento**
ImageDataGenerator ofrece muchos tipos de aumento de datos. Por ejemplo, un argumento es la rotación_rango. Esto rota cada imagen en una cantidad aleatoria que puede ser hasta el valor que especifique.

¿Sería sensato usar la rotación automática para este problema? ¿Por qué o por qué no?

El objetivo de este problema es saber si una imagen está en posición vertical o lateral. Rotar las imágenes moderadamente puede causar imágenes que no se sienten bien en ninguna de las categorías.

Dado que el aumento de datos afecta a las imágenes sin tocar las etiquetas, una rotación dramática significaría que algunas imágenes se usan para entrenar con la etiqueta incorrecta (por ejemplo, el generador rotará una imagen de lado y aún tendrá la etiqueta de estar en posición vertical)

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

# 3) Code
Fill in the missing pieces in the following code. We've supplied some boilerplate. You need to think about what ImageDataGenerator is used for each data source.

In [ ]:
#Especifica que tipos de ImageDataGenerator anterior se cargará en los datos de entrenamiento
train_generator = data_generator_with_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/images/train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')

#Especifica que tipos de ImageDataGenerator anterior se cargará en los datos de 
validation_generator = data_generator_no_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/images/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator, 
        epochs = 3,
        steps_per_epoch=19,
        validation_data=valid_generator)

In [ ]:
# Check your answer
q_3.check()

In [ ]:
# q_3.hint()
# q_3.solution()

# 4) Did Data Augmentation Help?
How could you test whether data augmentation improved your model accuracy?

¿Cómo podría probar si el aumento de los datos mejoró la precisión de su modelo

Crear train_generator usando data_generator_no_aug pero no cambies otros argumentos a train_generator.

Ejecuta el modelo y ve la precisión del resultado. Compara esto con la precisión que obtuviste cuando train_generator usó el aumento.

Nuestro conjunto de datos de validación es muy pequeño, así que hay un poco de suerte o de aleatoriedad en la puntuación exacta de cualquier ejecución del modelo. Los puntajes de validación serán más confiables cuando comiences a usar conjuntos de datos más grandes.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_4.solution()

In [ ]:
train_generator = data_generator_with_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')

# Specify which type of ImageDataGenerator above is to load in validation data
validation_generator = data_generator_no_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator,
        epochs = 3,
        steps_per_epoch=19,
        validation_data=validation_generator)

# Keep Going
You are ready for **[a deeper understanding of deep learning](https://www.kaggle.com/dansbecker/a-deeper-understanding-of-deep-learning/)**.


---
**[Deep Learning Home Page](https://www.kaggle.com/learn/deep-learning)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*